In [38]:
import math
import re

import pandas as pd

In [21]:
DATA_DIR = '../data'
WEIGHTS_FILE = f"{DATA_DIR}/weights/weights.csv"

In [28]:
df = pd.read_csv(WEIGHTS_FILE)
df = df.dropna()
df.head(3)

,Tag,Count,Key,Value,Weight
0,"('highway', 'service')",12556,highway,service,100.000000
1,"('highway', 'residential')",7917,highway,residential,63.399665
2,"('highway', 'footway')",6913,highway,footway,55.478403


In [29]:
grouped_df = df.groupby('Key')[['Value', 'Weight']].agg(list).reset_index()
grouped_df.head(10)

,Key,Value,Weight
0,crossing,"[uncontrolled, traffic_signals, marked, unmark...","[100.0, 92.8212478920742, 91.65261382799324, 2..."
1,footway,"[sidewalk, crossing]","[100.0, 1.0]"
2,highway,"[service, residential, footway, tertiary, prim...","[100.0, 63.39966528530443, 55.47840293273829, ..."
3,name,"[Phố Minh Khai, Đường Số 1 Times City, Đường S...","[100.0, 65.29520295202953, 53.42250922509225, ..."
4,oneway,"[yes, no]","[100.0, 1.0]"
5,surface,"[asphalt, concrete, paving_stones, concrete:pl...","[100.0, 9.15934065934066, 5.181662087912088, 4..."


In [42]:
get_tags = ""
tag_conditions = ""

for _, row in grouped_df.iterrows():
    get_tag = f"""
local {row['Key']} = way:get_value_by_key('{row['Key']}')
    """
    get_tags += get_tag
    for value, weight in zip(row['Value'], row['Weight']):        
        tag_condition = f"""
if {row['Key']} == '{value}' then
    result.forward_rate = result.forward_speed * {weight}
    result.backward_rate = result.backward_speed * {weight}
end
        """
        tag_conditions += tag_condition
        
custom_tag_weigths = f"""
function custom_tag_weights(profile, way, result, data)
{get_tags}
{tag_conditions}
end
"""

custom_tag_weigths = re.sub(r'\n\s*\n', '\n', custom_tag_weigths)

print(custom_tag_weigths)


function custom_tag_weights(profile, way, result, data)
local crossing = way:get_value_by_key('crossing')
local footway = way:get_value_by_key('footway')
local highway = way:get_value_by_key('highway')
local name = way:get_value_by_key('name')
local oneway = way:get_value_by_key('oneway')
local surface = way:get_value_by_key('surface')
if crossing == 'uncontrolled' then
    result.forward_rate = result.forward_speed * 100.0
    result.backward_rate = result.backward_speed * 100.0
end
if crossing == 'traffic_signals' then
    result.forward_rate = result.forward_speed * 92.8212478920742
    result.backward_rate = result.backward_speed * 92.8212478920742
end
if crossing == 'marked' then
    result.forward_rate = result.forward_speed * 91.65261382799324
    result.backward_rate = result.backward_speed * 91.65261382799324
end
if crossing == 'unmarked' then
    result.forward_rate = result.forward_speed * 22.70320404721754
    result.backward_rate = result.backward_speed * 22.7032040472175